# Some Nonconvex Example Problems

## Pooling Problem

A small distributor of wholesale milk creates custom blends of raw milk supplied from farms for delivery to customers. The distributor has found a new opportunity to improve profits by buying milk from farms located at a distance away key customers. The problem is that the distributor would need to combine the milk from the distant farms into a single tank for delivery by truck.

In [64]:
import pandas as pd

raw_milk_suppliers = pd.DataFrame({
    "Farm A": {"fat": 0.045, "price": 45.0, "location": "local"},
    "Farm B": {"fat": 0.030, "price": 42.0, "location": "local"},
    "Farm C": {"fat": 0.033, "price": 32.0, "location": "remote"},
    "Farm D": {"fat": 0.050, "price": 40.0, "location": "remote"}
}).T

customers = pd.DataFrame({
    "Customer A": {"fat": 0.040, "price": 52.0, "demand": 1000.0},
    "Customer B": {"fat": 0.030, "price": 48.0, "demand": 5000.0}
}).T

display(raw_milk_suppliers)
display(customers)

,fat,price,location
Farm A,0.045,45.0,local
Farm B,0.03,42.0,local
Farm C,0.033,32.0,remote
Farm D,0.05,40.0,remote


,fat,price,demand
Customer A,0.04,52.0,1000.0
Customer B,0.03,48.0,5000.0


1. Given the available supplies, both local and remote, what is the maximum profit the distributor can earn assuming there was no necessity to mix milk from the remote suppliers? Given that solution, would there be any problem with pooling the milk from the remote farms prior to delivery? 

2. How much profit is lost if the truck delivery is constrained to using just one of the remote farms? (note: this is an opportunity for disjunctive programming).

3. Reformulate the problem to help the distributor maximize profits by pooling supplies from the remote farms.

In [66]:
import pyomo.environ as pyo

m = pyo.ConcreteModel()

m.S = pyo.Set(initialize=raw_milk_suppliers.index)
m.C = pyo.Set(initialize=customers.index)
m.x = pyo.Var(m.S, m.C, domain=pyo.NonNegativeReals)

@m.Param(m.S)
def cost(m, s):
    return raw_milk_suppliers.loc[s, "price"]

@m.Param(m.S)
def fat(m, s):
    return raw_milk_suppliers.loc[s, "fat"]

@m.Param(m.C)
def price(m, c):
    return customers.loc[c, "price"]

@m.Param(m.C)
def fat_spec(m, c):
    return customers.loc[c, "fat"]

@m.Param(m.C)
def demand(m, c):
    return customers.loc[c, "demand"]

@m.Objective(sense=pyo.maximize)
def profit(m):
    return sum(m.x[s, c]*(m.price[c] - m.cost[s]) for s, c in m.S * m.C)

@m.Constraint(m.C)
def demand_limit(m, c):
    return sum(m.x[s, c] for s in m.S) <= m.demand[c]

@m.Expression(m.S, m.C)
def fat_shipped(m, s, c):
    return m.x[s, c]*m.fat[s]

@m.Constraint(m.C)
def fat_constraint(m, c):
    return  sum(m.fat_shipped[s, c] for s in m.S) >= sum(m.x[s, c]*m.fat_spec[c] for s in m.S)

pyo.SolverFactory('cbc').solve(m)

print(f"{m.profit():0.2f}")

soln = pd.DataFrame([[s, 
                      c, 
                      m.x[s, c](), 
                      raw_milk_suppliers.loc[s, "location"],
                      round(m.fat_shipped[s, c](), 0)
                     ] for s, c in m.S * m.C],
                    columns = ["supplier", "customer", "shipped", "location", "fat_shipped"])

display(soln)

pd.pivot_table(soln, index="supplier", columns=["customer"])


96705.88


,supplier,customer,shipped,location,fat_shipped
0,Farm A,Customer A,0.00000,local,0.0
1,Farm A,Customer B,0.00000,local,0.0
2,Farm B,Customer A,0.00000,local,0.0
3,Farm B,Customer B,0.00000,local,0.0
4,Farm C,Customer A,588.23529,remote,19.0
5,Farm C,Customer B,5000.00000,remote,165.0
6,Farm D,Customer A,411.76471,remote,21.0
7,Farm D,Customer B,0.00000,remote,0.0


fat_shipped               shipped           
customer  Customer A Customer B Customer A Customer B
supplier                                             
Farm A           0.0        0.0    0.00000        0.0
Farm B           0.0        0.0    0.00000        0.0
Farm C          19.0      165.0  588.23529     5000.0
Farm D          21.0        0.0  411.76471        0.0

$$
\begin{align*}
\sum_{}u_{s, p} & = \sum_{}v_{p, c} & \forall p \in \text{POOLS}
\end{align*}
$$